In the first notebook, a table was created that described the distance between points at every 25 meters on each modeling network segment. This tabe will be the guide to conflating the two networks. The conflation will start with divided highways and then conflate lower class segments. This process uses only the pandas module, and a variety of sorting methods to determine 'good' matches.

In [1]:
import pandas as pd
import os
import arcpy as ap
import csv 

modeling_network_file = os.getcwd() + '\\Modeling_Network_only_roads_test.shp'

path_to_data = os.getcwd() + '\\Data\\'

In [2]:
modeling_attributes_df = pd.read_csv(path_to_data + 'modeling_attributes.csv')
modeling_type = modeling_attributes_df.filter(['modeling_id', 'modeling_type'])
modeling_highways = modeling_type.query('modeling_type == 1')
list_of_modeling_highways = list(modeling_highways['modeling_id'])

del modeling_type, modeling_attributes_df

modeling_highways.head()

,modeling_id,modeling_type
0,0,1
2,2,1
16,16,1
302,302,1
469,469,1


In [3]:
osm_attributes_df = pd.read_csv(path_to_data + 'osm_attributes.csv')
osm_type = osm_attributes_df.filter(['osm_id', 'osm_type'])
osm_highways = osm_attributes_df[osm_attributes_df['osm_type'] == 'motorway']

list_of_osm_highways = list(osm_highways['osm_id'])
del osm_attributes_df, osm_type

osm_highways.head()

,osm_id,osm_name,osm_type,osm_bearing
0,0,Denver Boulder Turnpike,motorway,153.92
3,3,,motorway,201.76
4,4,,motorway,204.62
5,5,,motorway,350.27
125,125,Dwight D. Eisenhower Highway,motorway,359.07


In [4]:
matches = pd.read_csv(path_to_data + 'matches.csv')
matches = matches.filter(['modeling_id', 'osm_id', 'NEAR_RANK','distance'])

highway_modeling_matches = matches[matches['modeling_id'].isin(list_of_modeling_highways)]

highway_modeling_matches.head()

,modeling_id,osm_id,NEAR_RANK,distance
0,0,8163,1,4.858731
1,0,8120,2,41.678034
2,0,8163,1,7.919202
3,0,8120,2,45.998868
4,0,8163,1,8.581458


In [5]:
modeling_highways_df_matches_osm_highways_df = highway_modeling_matches[highway_modeling_matches['osm_id'].isin(
                                               list_of_osm_highways)]

del highway_modeling_matches, matches

modeling_highways_df_matches_osm_highways_df.head()

,modeling_id,osm_id,NEAR_RANK,distance
0,0,8163,1,4.858731
1,0,8120,2,41.678034
2,0,8163,1,7.919202
3,0,8120,2,45.998868
4,0,8163,1,8.581458


In [6]:
modeling_highways_df_matches_osm_highways_df = modeling_highways_df_matches_osm_highways_df.query('NEAR_RANK == 1')
modeling_highways_df_matches_osm_highways_df.head()

,modeling_id,osm_id,NEAR_RANK,distance
0,0,8163,1,4.858731
2,0,8163,1,7.919202
4,0,8163,1,8.581458
6,0,8163,1,8.474173
8,0,8163,1,8.366641


In [7]:
highway_matches = modeling_highways_df_matches_osm_highways_df[[
                        'modeling_id', 'osm_id', 'distance']].groupby([
                        'osm_id', 'modeling_id']).agg(['mean', 'max', 'min'])
highway_matches.head()

distance                     
                        mean        max       min
osm_id modeling_id                               
0      28156        1.548558   3.207750  0.137865
3      3176         0.141143   0.141143  0.141143
4      778          1.595584   4.209333  0.134585
       3176         2.626974   4.241345  1.076535
5      33937        7.506589  10.088710  3.018344

In [8]:
try:
    os.remove(path_to_data + 'highway_matches.csv')
except:
    pass

with open(path_to_data + 'highway_matches.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    
    writer.writerow(['osm_id', 'modeling_id', 'mean', 'max', 'min'])
    
    for each_match, details in highway_matches.iterrows():
        modeling_id, osm_id = each_match
        mean, maximum, minimum = details
        
        writer.writerow([modeling_id, osm_id, mean, maximum, minimum])
        
del highway_matches, modeling_highways_df_matches_osm_highways_df

In [9]:
modeling_highway_matches = pd.read_csv(path_to_data + 'highway_matches.csv')

modeling_attributes = pd.read_csv(path_to_data + 'modeling_attributes.csv')
osm_attributes = pd.read_csv(path_to_data + 'osm_attributes.csv')

In [10]:
highway_matches_with_attributes = pd.merge(modeling_highway_matches, 
                                                    modeling_attributes, 
                                                    on = 'modeling_id')

highway_matches_with_attributes = pd.merge(highway_matches_with_attributes, 
                                                        osm_attributes, 
                                                        on = 'osm_id')

del modeling_attributes, osm_attributes

highway_matches_with_attributes.head()

,osm_id,modeling_id,mean,max,min,modeling_name,modeling_type,modeling_bearing,osm_name,osm_type,osm_bearing
0,0,28156,1.548558,3.207750,0.137865,Us-36/Sh-270 Eb,1,153.73,Denver Boulder Turnpike,motorway,153.92
1,13529,28156,11.708245,11.708245,11.708245,Us-36/Sh-270 Eb,1,153.73,Denver Boulder Turnpike,motorway,132.78
2,13529,15337,5.493005,12.476778,0.227715,Us-36/Sh-270 Eb,1,128.50,Denver Boulder Turnpike,motorway,132.78
3,13529,15351,14.008784,17.439699,7.754535,Us-36 (Nw),1,129.03,Denver Boulder Turnpike,motorway,132.78
4,3,3176,0.141143,0.141143,0.141143,,1,202.61,,motorway,201.76


In [11]:
def getDifference(b1, b2):
    r = (b2 - b1) % 360.0
    # Python modulus has same sign as divisor, which is positive here,
    # so no need to consider negative case
    if r >= 180.0:
        r -= 360.0
    return r

In [12]:
try:
    os.remove(path_to_data + 'highway_matches_checked_with_angles.csv')
except:
    pass

with open(path_to_data + 'highway_matches_checked_with_angles.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)

    writer.writerow(['in_near'])
    
    for each_match in highway_matches_with_attributes.iterrows():
        osm_bearing = each_match[1].osm_bearing
        modeling_bearing = each_match[1].modeling_bearing
        if abs(getDifference(osm_bearing, modeling_bearing)) < 45:
            osm_id = each_match[1].osm_id
            modeling_id = each_match[1].modeling_id

            writer.writerow(['{}-{}'.format(str(modeling_id), str(osm_id))])

In [13]:
list_of_highway_matches = []

for each_match in highway_matches_with_attributes.iterrows():
        osm_bearing = each_match[1].osm_bearing
        modeling_bearing = each_match[1].modeling_bearing
        if abs(getDifference(osm_bearing, modeling_bearing)) < 45:
            osm_id = each_match[1].osm_id
            modeling_id = each_match[1].modeling_id
            
            list_of_highway_matches.append([modeling_id, osm_id])

In [14]:
try:
    os.remove(path_to_data + 'highway_matches.csv')
except:
    pass

matches = pd.read_csv(path_to_data + 'matches.csv')

with open(path_to_data + 'highway_matches.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['other_id','modeling_id', 'osm_id', 'from_x', 'from_y', 'near_x', 'near_y'])
    
    for each_match in matches.iterrows():
        if each_match[1].NEAR_RANK > 1:
            continue
        else:
            
            oid = each_match[0]
            modeling_id = int(each_match[1].modeling_id)
            osm_id = int(each_match[1].osm_id)
            if [modeling_id, osm_id] in list_of_highway_matches:
                from_x = each_match[1].FROM_X
                from_y = each_match[1].FROM_Y
                near_x = each_match[1].NEAR_X
                near_y = each_match[1].NEAR_Y
                writer.writerow([oid, modeling_id, osm_id, from_x, from_y, near_x, near_y])

In [ ]:
ap.Delete_management(path_to_data + 'highway_match_lines.shp')

ap.XYToLine_management(path_to_data + 'highway_matches.csv', 
                         path_to_data + 'highway_match_lines.shp',
                         'from_x',
                         'from_y',
                         'near_x',
                         'near_y',
                         'GEODESIC')

ap.AddGeometryAttributes_management(path_to_data + 'highway_match_lines.shp', 'LENGTH_GEODESIC')